In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path
import json

import numpy as np

home = str(Path.home())

RPLIB_DATA_PREFIX = os.environ.get("RPLIB_DATA_PREFIX")

if RPLIB_DATA_PREFIX is None: # Set default
    RPLIB_DATA_PREFIX=f'{home}/RPLib/data'
    
try:
    import pyrankability as pyrankability
    import pyrplib as pyrplib
except:
    print('Looking for packages in home directory')
    sys.path.insert(0,f"{home}") # Add the home directory relevant paths to the PYTHONPATH
    sys.path.insert(0,f"{home}/ranking_toolbox") # Add the home directory relevant paths to the PYTHONPATH
    sys.path.insert(0,f"{home}/RPLib") # Add the home directory relevant paths to the PYTHONPATH
    import pyrankability
    import pyrplib

import pandas as pd

Looking for packages in home directory


In [15]:
data = pyrplib.data.Data(RPLIB_DATA_PREFIX)

## Unprocessed

In [80]:
data.datasets_df.loc[data.datasets_df['Dataset Name'].str.contains('DOM')]

,Dataset ID,Dataset Name,Description,Type,Loader,Download links
75,76,DOM + Noise 10% to 90%,"'{\n ""high"": 5,\n ""low"": 1,\n ""make_u...",Structured Artificial,structured_artificial.base.Unprocessed,[https://raw.githubusercontent.com/IGARDS/stru...


In [21]:
unprocessed = pyrplib.dataset.load_unprocessed(78,data.datasets_df)

In [22]:
len(unprocessed.data())

5

## Example remove processed

In [6]:
collection = 'structured_artificial'

In [7]:
#155	76	46	transformers.process_D(data.loc[index,'D'])	D	structured_artificial	{}	14/03/2022 18:22:51

In [8]:
mask = data.processed_datasets_df['Collection'] == collection

In [9]:
data.processed_datasets_df.drop(data.processed_datasets_df.index[mask],inplace=True)

In [89]:
#data.save_processed_datasets()

## Processed

In [23]:
data.processed_datasets_df['Collection'].value_counts()

structured_artificial    270
marchmadness              54
lolib                     50
japan_io                   4
us_news_world_report       3
Name: Collection, dtype: int64

In [24]:
collection = 'structured_artificial'

In [25]:
mask = data.processed_datasets_df['Collection'] == collection

In [26]:
sum(mask)

270

In [27]:
data.processed_datasets_df[mask].head()

,Dataset ID,Source Dataset ID,Index,Command,Type,Collection,Options,Last Processed Datetime,Identifier,Link
111,412,76,0,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:51,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
112,413,76,1,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:52,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
113,414,76,2,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:53,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
114,415,76,3,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:54,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
115,416,76,4,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:55,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...


In [96]:
# only needed if adding new column data.processed_datasets_df['Identifier'] = ""

In [99]:
get_identifier = lambda item: "Threshold: "+str(item['percentage'])

In [100]:
index_10 = []
for index in data.processed_datasets_df[mask].index:
    dataset = data.processed_datasets_df.loc[index]
    dataset_id = dataset['Dataset ID']
    processed = pyrplib.dataset.ProcessedD.from_json(f"{RPLIB_DATA_PREFIX}/{collection}/{dataset_id}.json").load()
    unprocessed = pyrplib.dataset.load_unprocessed(processed.source_dataset_id,data.datasets_df)
    item = unprocessed.data().loc[dataset['Index']]
    if item['percentage'] == 10:
        index_10.append(index)
    data.processed_datasets_df.loc[index,'Identifier'] = get_identifier(item)

In [65]:
item = unprocessed.data().loc[59]
item

Outer Index                                                                0
number_matrices                                                           30
number_of_rows_columns                                                    20
low                                                                        0
high                                                                       5
percentage                                                                10
percent_links_to_remove                                                   10
make_unweighted                                                        False
Create code                def create():\n  D = domplusnoise(number_of_ro...
Inner Index                                                               29
D                              0  1  2  3  4  5  6  7  8  9  10  11  12  ...
Name: 59, dtype: object

In [101]:
data.processed_datasets_df.loc[index_10]

,Dataset ID,Source Dataset ID,Index,Command,Type,Collection,Options,Last Processed Datetime,Identifier,Link
111,412,76,0,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:51,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
112,413,76,1,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:52,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
113,414,76,2,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:53,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
114,415,76,3,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:54,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
115,416,76,4,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:55,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
116,417,76,5,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:56,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
117,418,76,6,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:57,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
118,419,76,7,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:58,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
119,420,76,8,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:59,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...
120,421,76,9,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},14/03/2022 18:22:60,Percentage of noise: 10,/home/jupyter-pander14/RPLib/data/structured_a...


In [102]:
data.processed_datasets_df.loc[mask,'Identifier'].value_counts()

Percentage of noise: 70    30
Percentage of noise: 30    30
Percentage of noise: 50    30
Percentage of noise: 20    30
Percentage of noise: 90    30
Percentage of noise: 60    30
Percentage of noise: 10    30
Percentage of noise: 40    30
Percentage of noise: 80    30
Name: Identifier, dtype: int64

In [103]:
data.processed_datasets_df.loc[mask,'Index'].value_counts()#.sort_values()

0      1
185    1
171    1
172    1
173    1
      ..
95     1
96     1
97     1
98     1
269    1
Name: Index, Length: 270, dtype: int64

In [104]:
data.save_processed_datasets()